In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

: 

In [8]:
import pymysql

DB_HOST = os.getenv("DB_HOST")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_SCHEMA = os.getenv("DB_SCHEMA")

con = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWORD, port=3306, database=DB_SCHEMA)

with pymysql.cursors.DictCursor(con) as empCursor:
    empCursor.execute("SELECT * FROM employees")
    # generator
    employees = empCursor.fetchall()

    rectified = []

    with pymysql.cursors.DictCursor(con) as attnCursor:
        for employee in employees:
            days = 0
            otHours = 0.0

            attnCursor.execute("SELECT * FROM attendance_record WHERE emp_id = %s AND date BETWEEN '2024-05-01' AND '2024-05-31'", employee['empId'])
            attendances = attnCursor.fetchall()
            for attendance in attendances:
                days = days + 1
                if attendance['ot_hours'] is not None:
                    otHours = otHours + (4.0 if attendance['ot_hours'] > 4.0 else attendance['ot_hours'])
            
            rectified.append({
                "emp_id": employee['empId'],
                "days": days,
                "ot_hours": otHours
            })

# C1351   # 0015 # 1920 

In [ ]:
len(rectified)

In [9]:
con = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWORD, port=3306, database=DB_SCHEMA)

with con.cursor() as cursor:
    for rect in rectified:
        cursor.execute('UPDATE attendance_history SET days = %s, ot_hours = %s WHERE emp_id = %s AND month = 5 AND year = 2024', (rect['days'], rect['ot_hours'], rect['emp_id']))

    con.commit()


# FTP connection


In [ ]:
import os
import json
from ftplib import FTP
import pymysql

from dotenv import load_dotenv

load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_SCHEMA = os.getenv("DB_SCHEMA")

FTP_HOST = os.getenv("FTP_HOST")
FTP_USER = os.getenv("FTP_USER")
FTP_PASSWORD = os.getenv("FTP_PASSWORD")
FTP_PORT = os.getenv("FTP_PORT")


con = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWORD, port=3306, database=DB_SCHEMA)

with pymysql.cursors.DictCursor(con) as cursor:
    dateList = [1,2,3,4,5,6,7,8,10,11,12]

    for i in range(len(dateList)):
        cursor.execute(f"SELECT * FROM attendance_record WHERE date = '2024-09-{dateList[i]}' AND emp_id = '1025'")
        employee = cursor.fetchone()

        if employee is not None:
            punchin = employee['punch_in']

            with FTP(FTP_HOST, FTP_USER, FTP_PASSWORD) as ftp:
                ftp.cwd(punchin)

                # log it
                log_file = None
                if os.path.exists("logs.txt"):
                    log_file = open("logs.txt", "a")
                else:
                    log_file = open("logs.txt", "w")
                
                log_file.write(str(dateList[i]))
                log_file.write("\n")

                for f in ftp.mlsd():
                    log_file.write(json.dumps(f))
                    log_file.write("\n")
                
                log_file.write("\n")
                log_file.close()

                try:
                    with open(f"image_at_{dateList[i]}-09-2024.jpg", "wb") as image:
                        ftp.retrbinary("RETR group_photo", image.write)
                except Exception as e:
                    print(e)
                
                print("DONE")

550 group_photo: No such file or directory
DONE
550 group_photo: No such file or directory
DONE
550 group_photo: No such file or directory
DONE
550 group_photo: No such file or directory
DONE
550 group_photo: No such file or directory
DONE
550 group_photo: No such file or directory
DONE
550 group_photo: No such file or directory
DONE
550 group_photo: No such file or directory
DONE
